In [ ]:
# === Cell 1: Install dependencies ===
!pip install \
    gradio==3.30.0 \
    pandas==1.5.3 \
    matplotlib==3.7.1 \
    seaborn==0.12.2 \
    scikit-learn==1.2.2 \
    numpy==1.23.5 \
    pillow==9.4.0 \
    opencv-python==4.7.0.72 \
    tensorflow==2.12.0



In [ ]:
# === Cell 2: Install & all imports ===

# 0) Pin inflect back to 6.x and install tensorflow-addons 0.22.0
!pip install -q inflect==6.0.4 tensorflow-addons==0.22.0
!pip install --upgrade tensorflow-probability==0.20.0

# 1) Standard library
import os, glob
import io, base64

# 2) Data & visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 3) TensorFlow & Keras
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization, Dropout, Dense
from tensorflow.keras.models import load_model
from tensorflow_addons.optimizers import AdamW
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow_probability as tfp

# 4) TensorFlow Add-Ons (for focal loss, etc.)
import tensorflow_addons as tfa

# 5) Scikit-learn utilities
from sklearn.utils.class_weight import compute_class_weight

# 6) OpenCV & PIL (for any image I/O)
import cv2
from PIL import Image
from IPython.display import display, HTML
from google.colab.patches import cv2_imshow
from google.colab import output, files
from google.colab import files


In [ ]:
# === Cell 8: Left-Aligned Webcam & Upload UI with Continuous Emotion Prediction ===

# --- Constants ---
VIDEO_WIDTH, VIDEO_HEIGHT = 640, 480
IMAGE_SIZE = 128 # model input height/width

# --- Load model & build single-trace inference fn ---
model = load_model("FER_MobileNetV2_best.h5")
@tf.function
def predict_fn(x):
    # run the model in inference mode, returns (batch, 7) tensor
    return model(x, training=False)

# --- Emotion labels & face detector ---
emotion_labels = {
    0: "Angry", 1: "Disgust", 2: "Fear",
    3: "Happy", 4: "Sad",     5: "Surprise",
    6: "Neutral"
}
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# --- Helper: detect faces, preprocess, predict, annotate ---
def annotate_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(face_rgb, (IMAGE_SIZE, IMAGE_SIZE))
        inp = preprocess_input(resized.astype("float32"))
        inp = tf.expand_dims(inp, axis=0)  # shape (1, IMAGE_SIZE, IMAGE_SIZE, 3)
        preds = predict_fn(inp).numpy()[0]
        idx = int(np.argmax(preds))
        label = emotion_labels.get(idx, "Unknown")
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return frame

# --- JavaScript UI: left-aligned video + bottom buttons ---
def start_webcam_ui():
    js = rf"""
    <script>
      const cid = 'emotion-container';
      document.getElementById(cid)?.remove();
      const c = document.createElement('div');
      c.id = cid;
      c.style.display = 'flex';
      c.style.flexDirection = 'column';
      c.style.alignItems = 'flex-start';
      c.style.margin = '10px';
      document.body.appendChild(c);

      const v = document.createElement('video');
      v.width = {VIDEO_WIDTH}; v.height = {VIDEO_HEIGHT}; v.autoplay = true;
      c.appendChild(v);

      navigator.mediaDevices.getUserMedia({{video:true}})
        .then(s => {{
          v.srcObject = s;
          setTimeout(() => {{
            const bar = document.createElement('div');
            bar.style.marginTop = '10px';
            c.appendChild(bar);

            const snap = document.createElement('button');
            snap.textContent = 'Capture Photo';
            bar.appendChild(snap);

            const upl = document.createElement('button');
            upl.textContent = 'Upload Image';
            bar.appendChild(upl);

            snap.onclick = () => {{
              const canvas = document.createElement('canvas');
              canvas.width = {VIDEO_WIDTH};
              canvas.height = {VIDEO_HEIGHT};
              canvas.getContext('2d').drawImage(v, 0, 0);
              const dataUrl = canvas.toDataURL('image/png');
              google.colab.kernel.invokeFunction('notebook.on_capture', [dataUrl], {{}});
            }};
            upl.onclick = () => {{
              google.colab.kernel.invokeFunction('notebook.on_upload', [], {{}});
            }};
          }}, 1000);
        }})
        .catch(e => alert('Webcam not accessible: ' + e));
    </script>
    """
    display(HTML(js))

# --- Callback: process a captured screenshot ---
def on_capture(dataUrl):
    header, encoded = dataUrl.split(",", 1)
    img = Image.open(io.BytesIO(base64.b64decode(encoded))).convert("RGB")
    frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # frame is already VIDEO_WIDTH×VIDEO_HEIGHT
    annotated = annotate_frame(frame)
    cv2_imshow(annotated)

# --- Callback: process an uploaded image ---
def on_upload():
    uploaded = files.upload()
    for fname in uploaded:
        img = Image.open(fname).convert("RGB")
        frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (VIDEO_WIDTH, VIDEO_HEIGHT))
        annotated = annotate_frame(frame)
        cv2_imshow(annotated)

# --- Register callbacks & launch UI ---
output.register_callback('notebook.on_capture', on_capture)
output.register_callback('notebook.on_upload', on_upload)
start_webcam_ui()
